In [5]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

ds = pd.read_csv("perfect_datasets/perfect_dataset_v3.csv")
ds_overview = pd.read_csv("perfect_datasets/perfect_dataset_v1_overview.csv")

cv = CountVectorizer(max_features=5000, stop_words="english")

vectors = cv.fit_transform(ds["tags"]).toarray()

# cv.get_feature_names_out().tolist()

similarity = cosine_similarity(vectors)
temp = pd.DataFrame()
temp["title"] = ds["title"].str.lower().str.split()
temp["title"] = temp["title"].apply(lambda x: " ".join(x))
temp["title"] = temp["title"].apply(lambda x: x.replace(":", ""))

movie_sugg = []

c = 0

def recommend(movie):
    global c
    # if(ds[(ds["title"].str.lower()).str.startswith(movie.lower())].empty):
    if(ds[ds["title"].str.lower() == movie.lower()].empty):
        for i in temp["title"]:
            if(movie.lower() in i) and c==0:
                movie_sugg.append(ds.iloc[temp[temp["title"] == i].index[0]].title)
        print("did you mean: ", movie_sugg, end="\n\n") if c==0 else print("No movie found")
        c = 1
        return
    elif(movie == ""):
        print("Enter a movie name")
        return
    # index = ds[(ds["title"].str.lower()).str.startswith(movie.lower())].index[0]
    index = ds[(ds["title"].str.lower()) == movie.lower()].index[0]
    distance = similarity[index]
    movie_list = sorted(list(enumerate(distance)), reverse=True, key=lambda x: x[1])[1:6]

    for i in movie_list:
        print("Movie Id: " + str(ds.iloc[i[0]].imdb_id))
        print("Movie Name: " + ds.iloc[i[0]].title)
        print("Movie Name: " + ds_overview.iloc[i[0]].overview, end="\n\n")
    
inp = str(input("Enter the movie name: "))
recommend(inp)

if movie_sugg != []:
        movie_re_input = str(input("Enter the movie name: "))
        recommend(movie_re_input)

MemoryError: Unable to allocate 7.72 GiB for an array with shape (32185, 32185) and data type float64